In [2]:
import pandas as pd
import json
import pprint
import copy

## Grammar

In [10]:
class Grammar:
    def __init__(self, path: str):
        self.nonterminals = []
        self.terminals = []
        self.productions = {}
        self.start = None

        with open(path, 'r') as file:
            try:
                data = json.load(file)
                self.nonterminals = data['NonTerminals']
                self.terminals = data['Terminals']
                self.productions = data['Productions']
                self.start = data['Start']
                self.setProductionNo()
            except Exception as e:
                print(e)
                print("Invalid JSON file")
                return
        
        print('Nonterminals:', self.nonterminals)
        print('Terminals:', self.terminals)
        print('Start:', self.start)
        self.getProductions()

    def getProduction(self, nonterminal: str):
        return self.productions[nonterminal]
    
    def isCFG(self):
        """
        A grammar is called Context-Free Grammar (CFG) if:
        1. Starting symbol is in the set of productions
        2. All left-hand sides of productions are nonterminals
        3. All right-hand sides of productions are strings of terminals and nonterminals (including ε)
        """
        if self.start not in self.nonterminals or self.start not in self.productions:
            return False

        for nonterminal, productions in self.productions.items():
            if nonterminal not in self.nonterminals:
                raise Exception(f"Invalid nonterminal: {nonterminal}")
                return False

            for production in productions:
                for symbol in production:
                    if not (symbol in self.nonterminals or symbol in self.terminals or symbol == 'ε'):
                        raise Exception(f"Invalid symbol: {symbol}")
                        return False
        return True

    def getProductions(self):
        print("Productions:")
        for nonterminal, productions in self.productions.items():
            production_str = f"   {nonterminal} -> {' | '.join(' '.join(prod) for prod in productions)}"
            print(production_str)

    def setProductionNo(self):
        i = 1
        self.productionNo = {}
        for nonterminal, productions_list in self.productions.items():
            for production in productions_list:
                key = (nonterminal, "".join(production))
                self.productionNo[key] = i
                i += 1

In [11]:
g1 = Grammar('Grammars/g1.json')
print('Is CFG:', g1.isCFG())
g1.productionNo

Nonterminals: ['S', 'A', 'B', 'C', 'D']
Terminals: ['+', '*', 'a', '(', ')']
Start: S
Productions:
   S -> B A
   A -> + B A | ε
   B -> D C
   C -> * D C | ε
   D -> ( S ) | a
Is CFG: True


{('S', 'BA'): 1,
 ('A', '+BA'): 2,
 ('A', 'ε'): 3,
 ('B', 'DC'): 4,
 ('C', '*DC'): 5,
 ('C', 'ε'): 6,
 ('D', '(S)'): 7,
 ('D', 'a'): 8}

In [5]:
g2 = Grammar('Grammars/g2.json')
print('Is CFG:', g1.isCFG())

Nonterminals: ['program', 'statement_list', 'statement', 'expression', 'term', 'factor', 'type', 'primitive', 'array_decl', 'declaration', 'ID_list', 'assignment', 'io', 'if_stmt', 'while_stmt', 'for_stmt', 'for_header', 'condition', 'relation']
Terminals: ['ID', 'CONST', 'int', 'str', 'list of', ':', ',', '=', '>>', '<<', 'if', '(', ')', 'do', 'else', 'end', 'while', 'for', 'in', '{', '}', ':', '(', ')', ':', ':', '==', '!=', '<', '<=', '>', '>=', '+', '-', '*', '/', '%', '\\n', '[', ']']
Start: program
Productions:
   program -> statement_list
   statement_list -> statement | statement \n statement_list
   statement -> declaration | assignment | io | if_stmt | while_stmt | for_stmt
   expression -> term | term + expression | term - expression
   term -> factor | factor * term | factor / term | factor % term
   factor -> ( expression ) | ID | CONST | ID [ expression ]
   type -> primitive | array_decl
   primitive -> int | str
   array_decl -> list of primitive
   declaration -> ID : 

## Parser

In [6]:
class Parser:
    def __init__(self, grammar: Grammar):
        self.grammar = grammar
        self.table = None
        self.tree = None
        self.first = {}
        self.follow = {}

    def First(self, verbose=False):
        """
        First(X) = {a | X => aα, a ∈ T ∪ {ε}, α ∈ (N ∪ T)*}
        """
        for terminal in self.grammar.terminals + ['ε']:
            self.first[terminal] = {terminal}

        for nonterminal in self.grammar.nonterminals:
            self.initialFirst(nonterminal)
        
        version = 0
        self.printFirst(version) if verbose else None
        modified = True
        while modified:
            modified = False
            for nonterminal in self.grammar.nonterminals:
                for production in self.grammar.getProduction(nonterminal):
                    symbol = production[0]
                    if symbol in self.grammar.nonterminals:
                        for first in self.first[symbol]:
                            if first not in self.first[nonterminal]:
                                self.first[nonterminal].add(first)
                                modified = True
            version+=1
            if modified:
                self.printFirst(version) if verbose else None

    def initialFirst(self, nonterminal: str):
        """
        Initialize first of nonterminal with first of its productions
        """
        self.first[nonterminal] = set()
        for production in self.grammar.getProduction(nonterminal):
            symbol = production[0]
            if symbol in self.grammar.terminals or symbol == 'ε':
                self.first[nonterminal].add(symbol)
            
            # for symbol in production:
            #     if symbol in self.grammar.terminals or symbol == 'ε':
            #         self.first[nonterminal].add(symbol)
            #         break
        
    def printFirst(self, version=None):
        print(f"[{version if version else '-'}] First:")
        for nonterminal in self.grammar.nonterminals:
            print(f"   {nonterminal} -> {self.first[nonterminal] if self.first[nonterminal] else '∅'}")

    def Follow(self, verbose=False):
        """
        Follow(X) = {a | S => αXaβ, a ∈ T ∪ {ε}, α, β ∈ (N ∪ T)*}
        """
        for nonterminal in self.grammar.nonterminals:
            self.follow[nonterminal] = set()
        
        self.follow[self.grammar.start].add('ε')

        follow_prev = copy.deepcopy(self.follow)
        version = 0
        self.printFollow(version) if verbose else None
        modified = True
        while modified:
            modified = False
            for nonterminal in self.grammar.nonterminals:
                for lhs, productions in self.grammar.productions.items(): # lhs -> [[a, B, y], [d, e, f]]]
                    for rhs in productions: # rhs -> [a, B, y]
                        for i, symbol in enumerate(rhs):
                            if nonterminal == symbol:
                                if i != len(rhs) - 1: # case: A -> aBy
                                    y = rhs[i+1]
                                    for first in self.first[y]:
                                        if first == 'ε': # case: A -> aBy, first(y) contains ε
                                            for follow in follow_prev[lhs]:
                                                if follow not in self.follow[nonterminal]:
                                                    self.follow[nonterminal].add(follow)
                                                    modified = True
                                        else:
                                            if first not in self.follow[nonterminal]:
                                                self.follow[nonterminal].add(first)
                                                modified = True
                                else: # case: A -> aB
                                    for follow in follow_prev[lhs]:
                                        if follow not in self.follow[nonterminal]:
                                            self.follow[nonterminal].add(follow)
                                            modified = True
            
            version+=1
            if modified:
                self.printFollow(version) if verbose else None
                follow_prev = copy.deepcopy(self.follow)

    def printFollow(self, version=None):
        print(f"[{version if version else '-'}] Follow:")
        for nonterminal in self.grammar.nonterminals:
            print(f"   {nonterminal} -> {self.follow[nonterminal] if self.follow[nonterminal] else '∅'}")


    def buildTable(self):
        self.First()
        self.Follow()
        self.printFirst()
        self.printFollow()
        self.table = pd.DataFrame(columns=self.grammar.terminals + ['$'], index=self.grammar.nonterminals + self.grammar.terminals + ['$'])
        self.table = self.table.fillna('-')

        for terminal in self.grammar.terminals:
            self.table.loc[terminal, terminal] = 'pop'
        self.table.loc['$', '$'] = 'acc'

        for nonterminal in self.grammar.nonterminals:
            for production in self.grammar.getProduction(nonterminal):
                first = self.first[production[0]]
                if 'ε' in first:
                    for b in self.follow[nonterminal]: # A -> α, ε ∈ First(α), b ∈ Follow(A) 
                        if b != 'ε':
                            self.table.loc[nonterminal, b] = self.getPair(nonterminal, production)
                        else:
                            self.table.loc[nonterminal, '$'] = self.getPair(nonterminal, production)
                    pass
                else: # A -> α, a ∈ First(α), a != ε
                    for a in first:
                        self.table.loc[nonterminal, a] = self.getPair(nonterminal, production) 


        print("Table:")
        display(pd.DataFrame(self.table))

    def getPair(self, nonterminal, production):
        return (''.join(production), self.grammar.productionNo[(nonterminal, ''.join(production))])



In [7]:
parser = Parser(g1)
parser.buildTable()

[-] First:
   S -> {'a', '('}
   A -> {'ε', '+'}
   B -> {'a', '('}
   C -> {'ε', '*'}
   D -> {'a', '('}
[-] Follow:
   S -> {'ε', ')'}
   A -> {'ε', ')'}
   B -> {'ε', ')', '+'}
   C -> {'ε', ')', '+'}
   D -> {'ε', ')', '+', '*'}
Table:


,+,*,a,(,),$
S,-,-,"(BA, 1)","(BA, 1)",-,-
A,"(+BA, 2)",-,-,-,"(ε, 3)","(ε, 3)"
B,-,-,"(DC, 4)","(DC, 4)",-,-
C,"(ε, 6)","(*DC, 5)",-,-,"(ε, 6)","(ε, 6)"
D,-,-,"(a, 8)","((S), 7)",-,-
+,pop,-,-,-,-,-
*,-,pop,-,-,-,-
a,-,-,pop,-,-,-
(,-,-,-,pop,-,-
),-,-,-,-,pop,-


## Tests

In [8]:
t1 = Grammar('Grammars/test1.json')
p1 = Parser(t1)
p1.buildTable()

Nonterminals: ['S', 'A', 'B']
Terminals: ['a', 'b', 'c', 'd']
Start: S
Productions:
   S -> a A B b
   A -> c | ε
   B -> d | ε
[-] First:
   S -> {'a'}
   A -> {'ε', 'c'}
   B -> {'ε', 'd'}
[-] Follow:
   S -> {'ε'}
   A -> {'ε', 'd'}
   B -> {'b'}
Table:


,a,b,c,d,$
S,"(aABb, 1)",-,-,-,-
A,-,-,"(c, 2)","(ε, 3)","(ε, 3)"
B,-,"(ε, 5)",-,"(d, 4)",-
a,pop,-,-,-,-
b,-,pop,-,-,-
c,-,-,pop,-,-
d,-,-,-,pop,-
$,-,-,-,-,acc


In [9]:
t2 = Grammar('Grammars/test2.json')
p2 = Parser(t2)
p2.buildTable()

Nonterminals: ['S', 'A', 'B']
Terminals: ['a', 'b']
Start: S
Productions:
   S -> A a A b | B b B a
   A -> ε
   B -> ε
[-] First:
   S -> {'ε'}
   A -> {'ε'}
   B -> {'ε'}
[-] Follow:
   S -> {'ε'}
   A -> {'a', 'b'}
   B -> {'a', 'b'}
Table:


,a,b,$
S,-,-,"(BbBa, 2)"
A,"(ε, 3)","(ε, 3)",-
B,"(ε, 4)","(ε, 4)",-
a,pop,-,-
b,-,pop,-
$,-,-,acc
